<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.6

In [10]:
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import io
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers
from sklearn.datasets import load_breast_cancer, load_boston

ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

In [11]:
#Dataframes
Wikipedia = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Wikipedia.csv') #text
Twitter = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Twitter.csv') #text

#Series
articles = Wikipedia['Text']
labels = Wikipedia['AI']
tweets = Twitter['tweet']
sentiments = Twitter['is_there_an_emotion_directed_at_a_brand_or_product']

#arrays
cancer_data = load_breast_cancer()['data']
diagnosis = dl.MulticlassOutput(load_breast_cancer()['target'])
housing_data = load_boston()['data']
prices = load_boston()['target']

#images
#uploaded = files.upload()
#data = zipfile.ZipFile(io.BytesIO(uploaded['IMGDS1.zip']), 'r')
#data.extractall()

In [6]:
#image directories... if any

# Parameter Grid Templates

rnn_grid = {'output_dim': [int], 'embedding': [vocab], 'nodes': [int], 'activation': [str], 
              'regularizer': [None or str], 'stacking': [bool], 'dropout': [bool], 'optimizer': [str], 
              'method': [str], 'bidirectional': [bool]} 

cnn_grid = {'output_dim': [int], 'base_filters':[int], 'kernel_size': [int], 'activation': [str], 
            'nblocks': [int], 'pool':[int], 'dropout': [bool], 'closer': [bool], 'optimizer': [str], 
            'metrics': [str]}

tc_grid = {'output_dim': [int], 'nodes': [int], 'activation': [str], 'regularizer': [None or str], 'stacking': [bool], 
               'dropout': [bool], 'nlayers': [int], 'closer': [bool], 'loss': [str], 'optimizer': [str]}

tr_grid = {'nodes': [int], 'activation': [str], 'regularizer': [None or str], 'stacking': [bool], 'dropout': [bool], 'nlayers': [int], 'closer': [bool], 
                'loss': [str], 'optimizer': [str], 'y_col': [str]}

In [16]:
def TCP(X, y):
  dl = DeepLearning()
  if len(X) != len(y):
    return "lengths {} and {} don't match.".format(len(X), len(y))
  avg = 0
  words = []
  for i in range(len(X)):
    avg += len(list(X)[i].split(' '))/len(X)
    word_list = list(X)[i].replace('\n', ' ').split(' ')
    for word in word_list:
      words.append(word.replace('.', '').replace(',', '').replace(' ', '').replace('"', '').replace(':', '').replace(';', '').replace('!', ''))
  pad = int(avg)
  text, labels = dl.ModelReadyText1(X, y, pad)
  return text, labels, len(np.unique(words))

In [20]:
def BigDawg(X, y, task, params, directories=[], classes=[], ts=150, epochs=50, batch_size=32, patience=5, regression=False):
  dl = DeepLearning()
  if task == 'NLP':
    X2, y2, embedding_dim = TCP(X, y)
    func = dl.RNN
  if task == 'CV':
    try:
      X2, _, y2, _ = dl.ModelReadyPixles(directories, classes, target_size=(ts, ts))
    except:
      return 'directories and classes cannot be empty if the task is CV'
    func = dl.CNN
  if task == 'TC':
    try:
      X2 = X
      y2 = dl.MulticlassOutput(y)
    except:
      X2 = np.array(X)
      y2 = dl.MulticlassOutput(np.array(y))
    func = dl.DeepTabularClassification
  if task == 'TR':
    try:
      X2 = X
      y2 = y
    except:
      X2 = np.array(X)
      y2 = dl.MulticlassOutput(np.array(y))
    func = dl.DeepTabularRegression
  return dl.CollectPerformance(params, func, X2, y2, epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)

In [22]:
rnn_grid = {'output_dim': [], 'embedding': [], 'nodes': [64], 'activation': ['tanh'], 'regularizer': [None, 'L1'],
            'stacking': [False], 'dropout': [False], 'optimizer': ['adam'], 'method': ['GRU'], 'bidirectional': [False, True]}

rnn_grid['output_dim'] = [2]

rnn_grid

{'activation': ['tanh'],
 'bidirectional': [False, True],
 'dropout': [False],
 'embedding': [],
 'method': ['GRU'],
 'nodes': [64],
 'optimizer': ['adam'],
 'output_dim': [2],
 'regularizer': [None, 'L1'],
 'stacking': [False]}

# Image dataset directory graveyard
copy and paste these into a code cell when working with the dataset

Dogs and Cats
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']